### Hyperparameter optimization with Genetic Algorithm

In [ ]:
from random import choice
from random import uniform
from random import randint
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import keras

In [ ]:
# Loading datasets
(X_train, y_train), (X_test, y_test)

# Preprocessing

# Scale images to the [0,1] range
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

# Make sure images have shape (28,28, 1)
X_train = np.expand_dims(x_train , -1)
X_test = np.expand_dims(x_test , -1)


num_classes = 5
input_shape = (575, 575, 1)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
def CNN_MODEL(f1, f2, a1, a2, k, d, opt, ep):
    model = keras.Sequential(
        [
        keras.Input(shape=(575, 575, 1)),
        layers.Conv2D(f1, kernel_size=(k,k), activation=a1),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(f2, kernel_size=(k,k), activation=a2),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(d),
        layers.Dense(5, activation='softmax')
        ]
    )

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(X_train, y_train, batch_size=128, epochs = ep, validation_split=0.1)

    return model


# initialization

def initialization():
    parameters={}

    # filter
    f1 = choice([32,64])
    parameters['f1']= f1
    f2 = choice([64,128])
    parameters['f2']= f2


    # kernel size
    k = choice([3,5])
    parameters['k'] = k

    # activation function
    a1 = choice(['relu','tanh', 'selu', 'elu'])
    parameters['a1']= a1
    a2 = choice(['relu','tanh', 'selu', 'elu'])
    parameters['a2']= a2

    # dropout
    d = round(uniform(0.1, 0.5) , 1)
    parameters['d'] = d

    # optimizer
    opt = choice(['adamax', 'adam', 'adadelta', 'adagrad'])
    parameters['opt'] = opt


    # number of epochs
    ep = randint(7,10)
    parameters["ep"] = ep

    return parameters



# define generation population function

def generate_population(n):
    population=[]
    for i in range(n):
        chromosome = initialization()
        population.append(chromosome)
        return population
     



# Fitness evaluation metric: Classification Accuracy

def fitness_evaluation(model):
    metrics = model.evaluate(X_test, y_test)
    return metrics[1]




def selection (pop_fitness):
    total = sum(pop_fitness)
    percentage = [round((x/total)*100 ) for x in pop_fitness]
    selection_wheel = []
    for pop_index, num in enumerate(percentage):
        selection_wheel.extend([pop_index]*num)
    parent1_ind = choice(selection_wheel)
    parent2_ind = choice(selection_wheel)

    return [parent1_ind, parent2_ind]






def crossover(parent1, parent2):
    child1 = {}
    child2 = {}

    child1['f1'] = choice([parent1['f1'], parent2['f1']])
    child1['f2'] = choice([parent1['f2'], parent2['f2']])

    child2['f1'] = choice([parent1['f1'], parent2['f1']])
    child2['f2'] = choice([parent1['f2'], parent2['f2']])

    child1['a1'] = parent1['a2']
    child2['a1'] = parent2['a2']


    child1['a2'] = parent2['a1']
    child2['a2'] = parent1['a1'] 

    child1['k'] = choice([parent1['k'], parent2['k']])
    child2['k'] = choice([parent1['k'], parent2['k']])

    child1['opt'] = parent2['opt']
    child2['opt'] = parent1['opt']

    child1['ep'] = parent1['ep']
    child2['ep'] = parent2['ep'] 

    return [child1, child2]


def mutation(chromosome):
    flag = randint(0,40)
    if flag <=20:
        chromosome['ep'] += randint(0,10)

    return chromosome




generations = 3
threshold = 90
num_pop = 4

population = generate_population(num_pop)

for g in range(generations):
    pop_fitness = []
    for chromosome in population:
        f1 = chromosome['f1']
        f2 = chromosome['f2']
        a1 = chromosome['a1']
        a2 = chromosome['a2']
        k = chromosome['k']
        d = chromosome['d']
        opt = chromosome['opt']
        ep = chromosome['ep']

    try:
        model = CNN_MODEL(f1, f2, a1, a2, k, d, opt, ep)
        acc = fitness_evaluation(model)
        print('Parameters:', chromosome)
        print('Accuracy:', round(acc,3))

    except:
        acc=0
        print('Parameters:', chromosome)
        print('Invalid parameters - Build fail')

    pop_fitness.append(acc)

print(pop_fitness)


parents_ind = selection(pop_fitness)
parent1 = population[parents_ind[0]]
parent2 = population[parents_ind[1]]

children = crossover(parent1, parent2)
child1 = mutation(children[0])
child2 = mutation(children[1])

population.append(child1)
population.append(child1)

print('Generation', g+1, 'Outcome')
if max(pop_fitness) >= threshold:
    print('Obtained desired accuracy: ', max(pop_fitness))
else:
    print('Maximum accuracy in generation{} : {}'.format(g+1, max(pop_fitness)))

first_min = min(pop_fitness)
first_min_ind = pop_fitness.index(first_min)
population.remove(population[first_min_ind])

second_min = min(pop_fitness)
second_min_ind = pop_fitness.index(second_min)
population.remove(population[second_min_ind])


Epoch 1/9
361/422 [========================>.....] - ETA: 7s - loss: 0.2769 - accuracy: 0.9185Parameters: {'f1': 32, 'f2': 128, 'k': 5, 'a1': 'elu', 'a2': 'relu', 'd': 0.5, 'opt': 'adam', 'ep': 9}
Invalid parameters - Build fail
Epoch 1/9
 96/422 [=====>........................] - ETA: 44s - loss: 0.6220 - accuracy: 0.8143Parameters: {'f1': 32, 'f2': 128, 'k': 5, 'a1': 'elu', 'a2': 'relu', 'd': 0.5, 'opt': 'adam', 'ep': 9}
Invalid parameters - Build fail
Epoch 1/9
 26/422 [>.............................] - ETA: 54s - loss: 1.3757 - accuracy: 0.5944

148/422 [=========>....................] - ETA: 44s - loss: 0.4859 - accuracy: 0.8544